In [1]:
datapath = "./data"
bestmodelpath = './model/bert_best_f1.pt'

In [5]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import time, datetime

In [6]:
from sqlalchemy import create_engine

# MySQL 서버 연결 정보 설정
host = '15.164.171.183'
user = 'mealddang'
password = 'test1234'
database = 'mealddang'

# MySQL에 연결하는 Connection String 생성
connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'

# 엔진 생성
engine = create_engine(connection_string)

In [7]:
#query = "SELECT * FROM md_review;"
#data = pd.read_sql(query, engine)

In [68]:
data = pd.read_csv('./data/naver_result_0305.csv', encoding='utf-8-sig')

C:\Users\USER\AppData\Local\Temp\ipykernel_18144\3342029167.py:1: DtypeWarning: Columns (44,57) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/naver_result_0305.csv', encoding='utf-8-sig')


In [70]:
data['rating'].replace('nan', np.nan, inplace=True)
data.dropna(subset=['rating'], inplace=True)

In [72]:
data['label'] = None

In [73]:
# rating 열을 숫자로 변환합니다.
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

# 숫자로 변환된 rating 열에서 4 이상인 행을 찾아서 label을 1로 설정합니다.
data.loc[data['rating'] >= 4, 'label'] = 1

# 숫자로 변환된 rating 열에서 2 이하인 행을 찾아서 label을 0으로 설정합니다.
data.loc[data['rating'] <= 2, 'label'] = 0

# label이 할당되지 않은 행을 제거합니다.
data = data.dropna(subset=['label'])

In [75]:
data = data[['label','body']].copy()

In [78]:
# DataFrame의 기본 인덱스를 사용하여 인덱스를 다시 설정합니다.
data.reset_index(drop=True, inplace=True)

In [79]:
# 열 이름 변경
data = data.rename(columns={'label': 'Rating', 'body': 'Review'})

In [80]:
data

,Rating,Review
0,0,"손님 놓치기 싫어서 지하1층으로 손님을 받을거면 동일한 서비스를 제공하세요 , 그동..."
1,1,맛있어요
2,1,김치찌개랑 계란말이 쨩
3,1,.
4,1,맛있어요 :)
...,...,...
13690,1,주차가돼요!
13691,1,좋아요
13692,1,깔끔하고 저녁엔 회사 사람들이 빠져나가서 조용하고 좋네요!!
13693,1,좋아요


In [81]:
data.Rating.value_counts()

Rating
1    13167
0      528
Name: count, dtype: int64

In [82]:
data.Review.str.len().describe()

count    13477.000000
mean        35.058322
std         50.083520
min          1.000000
25%          7.000000
50%         17.000000
75%         42.000000
max        400.000000
Name: Review, dtype: float64

In [83]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [84]:
tokenizer

BertTokenizer(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [85]:
test_data = data.copy()

In [ ]:
# train_data, valid_data = train_test_split(data, shuffle=True, stratify=data.Rating, random_state=42, test_size=0.2)

In [ ]:
# valid_data, test_data = train_test_split(valid_data, stratify=valid_data.Rating, random_state=42, test_size=0.5)

In [86]:
test_data.shape

(13695, 2)

In [ ]:
# train_data.shape, valid_data.shape, test_data.shape

((6691, 2), (836, 2), (837, 2))

In [87]:
np.bincount(test_data.Rating)

array([  528, 13167], dtype=int64)

In [ ]:
# np.bincount(train_data.Rating), np.bincount(valid_data.Rating), np.bincount(test_data.Rating)

(array([ 479, 6212]), array([ 60, 776]), array([ 60, 777]))

In [37]:
def bert_tokenizer(sent, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(text = sent, add_special_tokens=True, max_length=MAX_LEN, padding='max_length', return_attention_mask=True, truncation=True)

    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    token_type_ids = encoded_dict['token_type_ids']

    return input_ids, attention_masks, token_type_ids

In [38]:
def make_dataset(dataset, MAX_LEN):
    input_ids, attention_masks, token_type_ids = [], [], []
    for each_review in tqdm(dataset['Review']):
        input_id, attention_mask, token_type_id = bert_tokenizer(each_review, MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)

    tensordataset = TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(token_type_ids), torch.LongTensor(dataset['Rating'].values).unsqueeze(dim=1))
    return tensordataset

In [ ]:
# train_dataset = make_dataset(train_data, 256)

  0%|          | 0/6691 [00:00<?, ?it/s]

In [40]:
from torch.utils.data import TensorDataset

def make_dataset(dataset, max_length):
    input_ids = []
    attention_masks = []
    token_type_ids = []
    
    for text in dataset['Review']:
        # 텍스트를 토크나이징하고 인코딩합니다.
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'  # PyTorch tensors로 반환합니다.
        )
        
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
        token_type_ids.append(encoding['token_type_ids'])
    
    # 리스트를 PyTorch 텐서로 변환합니다.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    
    # 레이블을 PyTorch 텐서로 변환합니다.
    labels = torch.tensor(dataset['Rating'].values, dtype=torch.long).unsqueeze(dim=1)
    
    # TensorDataset을 생성합니다.
    tensordataset = TensorDataset(input_ids, attention_masks, token_type_ids, labels)
    
    return tensordataset

In [88]:
test_data['Rating'] = test_data['Rating'].astype(int)

In [89]:
test_data['Rating'].dtypes

dtype('int32')

In [93]:
test_data = test_data.dropna(subset=['Review'])

In [94]:
# valid_dataset = make_dataset(valid_data, 256)
test_dataset = make_dataset(test_data, 256)

In [95]:
test_dataset[0]

(tensor([   101,   9450, 108578,   9029,  18622,  12310,   9490,  12965,  12424,
           9706,  35506,  10759,  70450,  11467,   9450, 108578,  10622,   9322,
          10622,  41521,  14867,   9095,  75886,   9425, 104021,  11513,   9672,
          28000,  35506,  24982,  48549,    117,   8924,  93867,   9654,   8852,
          28578,   9531,  11467,  11018,   9102,  35465,  11903,  14040,   9521,
         118610,  48345,    119,   9405,  13890, 108578,  12030,  12508,  76854,
          37712,   9987,  13764,   9425, 119005,   9952,  14040,  23990,  28911,
           9061,  59894, 108436,   9885,  16758,  12605, 118867,  14153,   9689,
          14040,  11664,    117,   8924,  23811,  48387,  12092,   8982,  41693,
          86933,    122,  70450,   9583,  17342,  11287, 119192,  12965,  12424,
           9706,  35506,  15303,   9405,  61250,  12092,   9555,  32158,   9575,
         119351,  10739,  92564,  11489,  12092,   9034,  18622,  30005,  30936,
           9485, 100110,  21

In [ ]:
# train_dataset[0]

(tensor([   101,   9568,  12030, 118710,  12692,   9083,  41620,  22440,  10530,
           9367,  19855,  12310,   9685,  11664,   9638, 119023,   8895,    106,
            106,   9634,  21155,  19789,   9254,  10739,  40523,   8843,  45465,
          19789,   8982, 119022,  12508,   9523,  11903,    102,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,    

In [96]:
#train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
#valid_dataloader = DataLoader(valid_dataset) # default: batch_size=1, shuffle=False
test_dataloader = DataLoader(test_dataset)

In [49]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint_testing.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print("")
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [50]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=5e-6)
#loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')
epochs = 100
early_stopping = EarlyStopping(patience = 10, verbose = True, path=bestmodelpath)

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

177854978

In [51]:
device = torch.device("cuda")

In [52]:
!nvidia-smi

'nvidia-smi'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
train_loss_list, valid_loss_list, valid_accuracy_list, valid_f1_list = [], [], [], []
optimizer.zero_grad()
for epoch in range(epochs):
    t0 = time.time()
    train_pred_list = []
    train_loss, valid_loss, valid_accuracy = 0.0, 0.0, 0.0
    epoch_loss = 0.0
    model.train()
    optimizer.zero_grad()
    for batch in train_dataloader:
        b_input_ids, b_attention_masks, b_token_type_ids, b_labels = tuple(t.to(device) for t in batch)
        out = model(b_input_ids, b_attention_masks, b_token_type_ids, labels=b_labels)
        loss, logits = out.loss, out.logits
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()
        #pred = torch.argmax(F.softmax(logits, dim=0), dim=1).unsqueeze(dim=1).cpu()
        #pred_list.append(pred)
        #epoch_accuracy += (pred==b_labels).cpu().numpy().mean()

        optimizer.zero_grad()

    train_loss = float(epoch_loss / len(train_dataloader))
    #train_accuracy = float(epoch_accuracy / len(train_dataloader))

    train_loss_list.append(train_loss)
    #train_accuracy_list.append(train_accuracy)

    valid_pred_list, valid_real_list = [], []
    valid_accuracy, valid_f1, epoch_accuracy, epoch_loss = 0.0, 0.0, 0.0, 0.0
    with torch.no_grad():
        model.eval()
        for batch in valid_dataloader:
            b_input_ids, b_attention_masks, b_token_type_ids, b_labels = tuple(t.to(device) for t in batch)
            out = model(b_input_ids, b_attention_masks, b_token_type_ids, labels=b_labels)
            loss, logits = out.loss, out.logits
            epoch_loss += loss.item()

            pred = torch.argmax(logits).cpu()
            valid_pred_list.append(pred)
            valid_real_list.append(b_labels.squeeze(dim=0).squeeze(dim=0).cpu())
            #epoch_accuracy += (pred==b_labels).cpu().numpy()


        valid_loss = float(epoch_loss / len(valid_dataloader))
        #valid_accuracy = float(epoch_accuracy / len(valid_dataloader))
        valid_accuracy = accuracy_score(valid_real_list, valid_pred_list)
        valid_f1 = f1_score(valid_real_list, valid_pred_list)

        valid_loss_list.append(valid_loss)
        valid_accuracy_list.append(valid_accuracy)
        valid_f1_list.append(valid_f1)

        print(f"EPOCH: {epoch}  ||  Elapsed: {format_time(time.time()-t0)}.")
        print(f"   Train_loss: {train_loss:.4f}  ||  Valid_acc: {valid_accuracy:.4f} | Valid_f1: {valid_f1:.4f} | Valid_loss: {valid_loss:.4f}")

        early_stopping(valid_loss, model)
        print("")
        if early_stopping.early_stop:
            print("Early stopping")
            break

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


EPOCH: 0  ||  Elapsed: 0:04:39.
   Train_loss: 0.2115  ||  Valid_acc: 0.9486 | Valid_f1: 0.9728 | Valid_loss: 0.1656

Validation loss decreased (inf --> 0.165599).  Saving model ...



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


EPOCH: 1  ||  Elapsed: 0:04:44.
   Train_loss: 0.1222  ||  Valid_acc: 0.9522 | Valid_f1: 0.9744 | Valid_loss: 0.1487

Validation loss decreased (0.165599 --> 0.148699).  Saving model ...



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

EPOCH: 10  ||  Elapsed: 0:04:44.
   Train_loss: 0.0166  ||  Valid_acc: 0.9545 | Valid_f1: 0.9759 | Valid_loss: 0.2018
EarlyStopping counter: 8 out of 10



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


EPOCH: 11  ||  Elapsed: 0:04:44.
   Train_loss: 0.0128  ||  Valid_acc: 0.9533 | Valid_f1: 0.9748 | Valid_loss: 0.1680
EarlyStopping counter: 9 out of 10



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


EPOCH: 12  ||  Elapsed: 0:04:44.
   Train_loss: 0.0097  ||  Valid_acc: 0.9557 | Valid_f1: 0.9761 | Valid_loss: 0.2173
EarlyStopping counter: 10 out of 10

Early stopping


In [ ]:
print(classification_report(valid_real_list, valid_pred_list)) # 이건 12번째 epoch의 valid 결과

              precision    recall  f1-score   support

           0       0.68      0.73      0.70        60
           1       0.98      0.97      0.98       776

    accuracy                           0.96       836
   macro avg       0.83      0.85      0.84       836
weighted avg       0.96      0.96      0.96       836



In [97]:
model.load_state_dict(torch.load(bestmodelpath, map_location=torch.device('cpu')))
device = torch.device('cpu')
test_pred_list, test_real_list = [], []
test_accuracy, test_f1 = 0.0, 0.0
with torch.no_grad():
    model.eval()
    for batch in tqdm(test_dataloader, desc="Testing"):
        b_input_ids, b_attention_masks, b_token_type_ids, b_labels = tuple(t.to(device) for t in batch)
        out = model(b_input_ids, b_attention_masks, b_token_type_ids, labels=b_labels)
        loss, logits = out.loss, out.logits

        pred = torch.argmax(logits).cpu()
        test_pred_list.append(pred)
        test_real_list.append(b_labels.squeeze(dim=0).squeeze(dim=0).cpu())

    test_accuracy = accuracy_score(test_real_list, test_pred_list)
    test_f1 = f1_score(test_real_list, test_pred_list)

Testing:   0%|          | 0/13477 [00:00<?, ?it/s]

In [98]:
print(f"Test_acc: {test_accuracy:.4f} | Test_f1: {test_f1:.4f}") # 이건 best epoch의 test 결과

Test_acc: 0.9682 | Test_f1: 0.9834


In [99]:
print(classification_report(test_real_list, test_pred_list))

              precision    recall  f1-score   support

           0       0.58      0.67      0.62       526
           1       0.99      0.98      0.98     12951

    accuracy                           0.97     13477
   macro avg       0.78      0.82      0.80     13477
weighted avg       0.97      0.97      0.97     13477

